In [2]:
import coiled
import dask.bag as db
from dask.distributed import Client

In [5]:
cluster = coiled.Cluster(
    name="github-json",
    software="matthew-powers/my_env", 
    n_workers=10,
)

Output()

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-matthew24-147684-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-147684-firewall -> coiled-dask-matthew24-147684-firewall]
Created FW rules: coiled-dask-matthew24-147684-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-147684-cluster-firewall -> coiled-dask-matthew24-147684-cluster-firewall]
Created scheduler VM: coiled-dask-matthew24-147684-scheduler (type: t3.medium, ip: ['44.200.160.149'])


In [6]:
client = Client(cluster)

In [16]:
client

<Client: 'tls://10.4.7.85:8786' processes=10 threads=20, memory=76.67 GiB>

In [7]:
import datetime

start = datetime.datetime.strptime("01-01-2015", "%d-%m-%Y")
end = datetime.datetime.strptime("31-12-2015", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
prefix = "https://data.gharchive.org/"
filenames = []
for date in date_generated:
    for hour in range(1,24):
        filenames.append(prefix + date.strftime("%Y-%m-%d") + '-' + str(hour) + '.json.gz')

In [8]:
# Check filenames look ok
filenames[:100:10]

['https://data.gharchive.org/2015-01-01-1.json.gz',
 'https://data.gharchive.org/2015-01-01-11.json.gz',
 'https://data.gharchive.org/2015-01-01-21.json.gz',
 'https://data.gharchive.org/2015-01-02-8.json.gz',
 'https://data.gharchive.org/2015-01-02-18.json.gz',
 'https://data.gharchive.org/2015-01-03-5.json.gz',
 'https://data.gharchive.org/2015-01-03-15.json.gz',
 'https://data.gharchive.org/2015-01-04-2.json.gz',
 'https://data.gharchive.org/2015-01-04-12.json.gz',
 'https://data.gharchive.org/2015-01-04-22.json.gz']

In [11]:
import json

# read in json data
records = db.read_text(filenames).map(json.loads)

In [12]:
# filter out only PushEvents
push = records.filter(lambda record: record["type"] == "PushEvent")

In [13]:
# define function that extracts relevant data
def process(record):
    try:
        for commit in record["payload"]["commits"]:
            yield {
                "user": record["actor"]["login"],
                "repo": record["repo"]["name"],
                "created_at": record["created_at"],
                "message": commit["message"],
                "author": commit["author"]["name"],
            }
    except KeyError:
        pass

In [14]:
# process into single commit entries
processed = push.map(process)

In [22]:
processed.flatten().take(2)

({'user': 'soumith',
  'repo': 'soumith/fbcunn',
  'created_at': '2015-01-01T01:00:00Z',
  'message': 'back to old structure, except lua files moved out',
  'author': 'Soumith Chintala'},
 {'user': 'soumith',
  'repo': 'soumith/fbcunn',
  'created_at': '2015-01-01T01:00:00Z',
  'message': '...',
  'author': 'Soumith Chintala'})

In [24]:
processed.npartitions

8372

In [28]:
processed.flatten().map(json.dumps).take(2)

('{"user": "soumith", "repo": "soumith/fbcunn", "created_at": "2015-01-01T01:00:00Z", "message": "back to old structure, except lua files moved out", "author": "Soumith Chintala"}',
 '{"user": "soumith", "repo": "soumith/fbcunn", "created_at": "2015-01-01T01:00:00Z", "message": "...", "author": "Soumith Chintala"}')

In [29]:
processed.flatten().map(json.dumps).to_textfiles("s3://coiled-datasets/github-archive/github-archive-2015-json/*.json.gz", compression="gzip")

['coiled-datasets/github-archive/github-archive-2015-json/0000.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0001.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0002.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0003.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0004.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0005.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0006.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0007.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0008.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0009.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0010.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0011.json.gz',
 'coiled-datasets/github-archive/github-archive-2015-json/0012.json.gz',
 'coiled-datasets/github-archive/github-archive-201

In [17]:
# flatten and cast to dataframe
df = processed.flatten().to_dataframe()

In [19]:
df.head(2)

,user,repo,created_at,message,author
0,soumith,soumith/fbcunn,2015-01-01T01:00:00Z,"back to old structure, except lua files moved out",Soumith Chintala
1,soumith,soumith/fbcunn,2015-01-01T01:00:00Z,...,Soumith Chintala
